In [1]:
import bson
import pandas as pd
import datetime
from bson.objectid import ObjectId
from itertools import groupby
from operator import itemgetter


In [2]:
with open('test/datarows.bson', 'rb') as f:
    datarows = bson.decode_all(f.read())


with open('test/devices.bson', 'rb') as f:
    devices = bson.decode_all(f.read())

import bson

with open('test/animals.bson', 'rb') as f:
    animals = bson.decode_all(f.read())

In [3]:
# Crear un diccionario donde la clave es el "deviceMACAddress" y el valor es el diccionario completo del dispositivo correspondiente
device_dict = {}
for device in devices:
    device_dict[device["deviceMACAddress"]] = device

# Crear una nueva lista de datos fusionando los datos de datarows y devices
merged_data = []
for row in datarows:
    if row["UUID"] in device_dict:
        device_data = device_dict[row["UUID"]]
        merged_row = {**row, **device_data} # Fusionar los dos diccionarios en uno solo
        merged_data.append(merged_row)

In [4]:
#MERGE CON ANIMALS

for row in merged_data:
    animal_id = row.get('deviceAnimalID')
    animal = next((a for a in animals if a.get('_id') == animal_id), None)
    if animal:
        row['animal'] = animal
    else:
        row['animal'] = None

In [5]:
#TO DATE TIME

for d in merged_data:
    timestamp_str = d['dataRowData']['timestamp']
    try:
        timestamp_obj = pd.to_datetime(timestamp_str)
    except ValueError:
        try:
            timestamp_obj = pd.to_datetime(timestamp_str, format='%Y-%d-%mT%H:%M:%S')
        except ValueError:
            timestamp_obj = timestamp_obj
    d['dataRowData']['timestamp'] = timestamp_obj

In [6]:
def gps(desired_id: str, date_str: str):
    # Convertir la fecha en un objeto datetime.date
    date_obj = datetime.datetime.strptime(date_str, "%Y, %m, %d").date()
    
    # Convertir el string de desired_id en un objeto ObjectId
    desired_id_obj = ObjectId(desired_id)
    
    #FILTRO POR ID CAMPO y DATAROWTYPE = GPS
    gps_data = [element for element in merged_data if element.get("dataRowType") == "GPS" and desired_id_obj in element["animal"]["animalSettlement"]]
    #FILTRO POR DIA
    filtered_data = [d for d in gps_data if d["dataRowData"]["timestamp"] == date_obj]


    # Contar UUIDs/COLLARES únicos ese día
    unique_uuids = set(d['UUID'] for d in filtered_data)
    

    # CREO DICCIONARIO KEYS= COLLAR, INFO LAT LONG DATE
    gps_data_by_uuid = {}
    for uuid in unique_uuids:
        filtered_gps_data_animal = [d for d in filtered_data if d['UUID'] == uuid]
        gps_data_dict = {'timestamp': [], 'latitud': [], 'longitud': []}
        for d in filtered_gps_data_animal:
            gps_data_dict['timestamp'].append(d['dataRowData']['timestamp'])
            gps_data_dict['latitud'].append(d['dataRowData']['lat'])
            gps_data_dict['longitud'].append(d['dataRowData']['lng'])
        gps_data_by_uuid[uuid] = gps_data_dict
 

    
    return (gps_data_by_uuid)


#TRAE CANTIDAD DE COLLARES, DENTRO DATE Y LOC, PARAMETRO STR ID CAMPO FECHA, STR = EJ "2022, 9, 21"


In [7]:
collardicc = gps("620e6e5e60543d0026a01f0e","2022, 9, 21")

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_8088\2991580004.py:11: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  filtered_data = [d for d in gps_data if d["dataRowData"]["timestamp"] == date_obj]


In [8]:
collardicc.keys()

dict_keys(['0004A30B00F82E9C', '0004A30B00F88969', '0004A30B00EE959D', '0004A30B00F825B7', '0004A30B00F82BD2'])

In [24]:
type(collardicc["0004A30B00F825B7"])

dict

In [25]:
print(collardicc["0004A30B00F825B7"]["timestamp"][0])

print(collardicc["0004A30B00F825B7"]["latitud"][0])

print(collardicc["0004A30B00F825B7"]["longitud"][0])

2022-09-21 00:53:02
-32.85475433333333
-63.6427645


In [11]:
def caravanas(desired_id: str, date_str: str):
    # Convertir la fecha en un objeto datetime.date
    date_obj = datetime.datetime.strptime(date_str, "%Y, %m, %d").date()

    # Convertir el string de desired_id en un objeto ObjectId
    desired_id_obj = ObjectId(desired_id)

    # Filtrar por ID CAMPO y DATAROWTYPE = CARAVANA
    beacon_data = [element for element in merged_data if element.get("dataRowType") == "BEACON" and desired_id_obj in element["animal"]["animalSettlement"]]

    # Filtrar por día
    filtered_dataB = [d for d in beacon_data if d["dataRowData"]["timestamp"].date() == date_obj]

    
    # Contar UUIDs/COLLARES únicos ese día
    unique_uuids = set(d['UUID'] for d in filtered_dataB)

    collar_data = {}
    collar_list = set(d['UUID'] for d in filtered_dataB)

    for collar in collar_list:
        #FILTRO POR COLLAR
        beacon_data1 = [element for element in filtered_dataB if element.get("UUID") == collar]
        
        data_list = []
        
        for d in beacon_data1:
            date = d['dataRowData']['timestamp']
            mac = d['dataRowData']['mac']
            data_list.append({'date': date, 'mac': mac})
        
        
        # Ordenar la lista por fecha
        data_list_sorted = sorted(data_list, key=itemgetter('date'))
        
        # Agrupar por hora
        grouped_data = []
        for hour, group in groupby(data_list_sorted, lambda x: x['date'].strftime('%Y-%m-%d %H')):
            macs = set()
            for item in group:
                macs.add(item['mac'])
            grouped_data.append({'hour': hour, 'macs': macs})
        
        collar_data[collar] = grouped_data

    return collar_data


#TRAE CARAVANAS EN BASE A COLLARES POR ID CAMPO Y FECHA

In [12]:
collardicc1 = caravanas("620e6e5e60543d0026a01f0e","2022, 9, 21")

In [13]:
collardicc1.keys()

dict_keys(['0004A30B00F82E9C', '0004A30B00EE959D', '0004A30B00F825B7', '0004A30B00F82BD2'])

In [14]:
collardicc1["0004A30B00F825B7"]

[{'hour': '2022-09-21 12', 'macs': {'A7AA2D0FF0F3'}},
 {'hour': '2022-09-21 18', 'macs': {'7F93E1ED74CA'}},
 {'hour': '2022-09-21 20', 'macs': {'86395D2360F6'}}]

In [15]:
collardicc1["0004A30B00F825B7"][1]

{'hour': '2022-09-21 18', 'macs': {'7F93E1ED74CA'}}

In [16]:
collardicc1["0004A30B00F82E9C"][0]["macs"]

{'1D96FE9FFEF4', '3B415B321DDA', 'F356641DB6D3'}

In [17]:
import bson
from bson import ObjectId

In [18]:

with open('test/settlements.bson', 'rb') as f:
    settlements = bson.decode_all(f.read())

with open('test/plots.bson', 'rb') as f:
    plots = bson.decode_all(f.read())

In [19]:


def obtener_geopointsp(desired_id: str):
    
    # Convertir el string de desired_id en un objeto ObjectId
    desired_id_obj = ObjectId(desired_id)
    
    # Obtener lista de IDs de parcelas
    lista_plots = []
    for a in settlements:
        if a["_id"] == desired_id_obj:
            lista_plots.extend(a["plots"])
            
    # Crear lista para guardar los resultados
    lista_resultados = []
    
    # Recorrer lista de IDs de parcelas y buscar cada una en la colección de parcelas
    for id_parcela in lista_plots:  # La lista de IDs está dentro de otra lista
        for plot in plots:
            if plot["_id"] == id_parcela:
                # Obtener geopoints de la parcela y agregarlos a la lista de resultados
                geopoints = plot["geoPoints"]
                lista_resultados.append({"_id": plot["_id"], "geopoints": geopoints})
    
    # Devolver lista de resultados
    return lista_resultados



#TRAE EN BASE A ID CAMPO, LA CANTIDAD DE LOTES QUE LO CONFORMAN, Y SUS GEOPOINTS

In [20]:
geopoints = obtener_geopointsp("620e6e5e60543d0026a01f0e")

In [29]:
geopoints[0]["_id"]

ObjectId('63b55f4b973f9f35c0c5b37d')

In [41]:
geopoints[1]["geopoints"][0]["lng"]

-63.64061

In [49]:
geopoints[3]  #CADA ELEMENTO ES UN PLOT

{'_id': ObjectId('63e4f73507c91900306018e6'),
 'geopoints': [{'_id': ObjectId('63e4f73507c91900306018e7'),
   'lat': -32.848689076537156,
   'lng': -63.63217129296387},
  {'_id': ObjectId('63e4f73507c91900306018e8'),
   'lat': -32.85470979619781,
   'lng': -63.633501668635255},
  {'_id': ObjectId('63e4f73507c91900306018e9'),
   'lat': -32.85470979619781,
   'lng': -63.629253049555665},
  {'_id': ObjectId('63e4f73507c91900306018ea'),
   'lat': -32.85225829498449,
   'lng': -63.62732185906494},
  {'_id': ObjectId('63e4f73507c91900306018eb'),
   'lat': -32.84915776997614,
   'lng': -63.62710728234375}]}

In [22]:
from pymongo import MongoClient

def obtener_geopoints(object_ids):
    client = MongoClient()
    db = client['nombre_de_tu_base_de_datos']
    collection = db['nombre_de_tu_colección']
    
    result = []
    for object_id in object_ids:
        query = {"_id": object_id}
        projection = {"_id": 1, "virtualFenceGeoPoints": 1}
        doc = collection.find_one(query, projection)
        if doc is not None:
            geopoints = doc.get("virtualFenceGeoPoints")
            if geopoints is not None:
                result.append({"_id": str(object_id), "geopoint": geopoints})
    
    return result
